In [8]:
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
dat = pd.read_csv(r"C:\Users\OneDrive - SAP SE\Machine Learning\Dataset\train.csv")

In [9]:
dat= dat[:20]
dat.info()
dat['author'].isnull().sum()
dat['author'].fillna('UNK',inplace = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20 non-null     int64 
 1   title   20 non-null     object
 2   author  18 non-null     object
 3   text    20 non-null     object
 4   label   20 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 928.0+ bytes


In [10]:
dat['author'].isnull().sum()

0

In [11]:
dat.drop(columns='id',inplace=True)
dat['total'] = dat[['title','text']].apply(lambda x: ' '.join(x),axis = 1 )
dat.drop(columns=['title','text'],inplace=True)

In [23]:
vocab_size = 50
encoded_docs = list()
# encoded_docs = [one_hot(d, vocab_size) for d in docs]
for d in dat['total']:
    encoded_docs.append(one_hot(d, vocab_size))

print(len(encoded_docs[0]))

839


In [42]:
labels = dat['label'].to_numpy()

In [40]:
max_length = 32
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post',truncating='post' )

In [37]:
padded_docs

array([[39, 12, 27, 36, 34, 15, 18, 34, 47,  8,  1, 29, 48, 29, 39, 12,
        27, 36, 34, 15, 18, 34, 47,  8,  1, 29, 48, 29,  2, 10,  6,  5],
       [25, 23, 45, 49, 33,  5, 18, 30, 10, 42, 13,  6,  4, 31,  5, 13,
        39, 38,  2, 27, 33, 20,  1, 48, 34, 13,  1,  7, 23, 45, 16, 13],
       [ 2, 13,  9, 16, 42, 43, 27,  2, 13,  9, 16, 42, 43, 27, 23, 49,
        45, 13, 17, 48, 23, 13, 29,  1, 22, 49, 38, 24, 48, 38, 32, 29],
       [ 7, 13,  1, 33, 30, 47, 17, 46, 24, 44, 24,  7, 13,  1, 33, 30,
        47, 17, 46, 24, 44, 13, 48,  3, 48, 13, 46, 21,  1,  2, 22, 36],
       [26, 33,  3, 23,  1,  2, 14, 32, 33, 19, 18, 49, 23, 24, 16, 40,
        26, 33, 49, 24, 26, 18, 18, 23, 33, 11, 48, 14, 18, 14, 33, 45],
       [37,  4,  9,  5, 44, 25, 20,  5, 12, 28, 43,  8, 42,  4, 47, 41,
        25,  6, 30, 33, 19, 14, 34, 37,  4, 46, 13, 24,  4, 40, 33, 34],
       [31, 31,  4, 23,  4,  9, 49,  3, 40,  3, 18, 30,  3,  5, 41, 29,
        40, 10, 36,  2, 40, 24,  2, 34, 20, 30,  7, 20, 41

In [32]:
type(padded_docs)

numpy.ndarray

In [33]:
vocab_size = 5000
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 32)            160000    
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 161,025
Trainable params: 161,025
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 32)            160000    
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 161,025
Trainable params: 161,025
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
model.fit(padded_docs, labels, epochs=50, verbose=0)

In [44]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)

In [46]:
print(loss)
print(accuracy)

0.2567594647407532
1.0
